In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-


SOURCE_FILES = [('01.txt', 'Test 1'),
                ('02.txt', 'Test 2'),
                ('03.txt', 'Test 3'),
                ('04.txt', 'Test 4'),
                ('05.txt', 'Test 5'),
                ('hamlet.txt', 'Гамлет, принц датский'), 
                ('anna_karenina.txt', 'Анна Каренина'),
                ('digits_of_pi.txt', '100 000 знаков числа пи'),
                ('don_quijote.txt', 'Дон Кихот Ламанчский'),
               ]

IS_READY_METHOD = [('huffman', False),
                   ('lzw', True),
                  ]


import os
from math import ceil
import pickle
import pprint


def huffman_encode(text):
    # В параметре text передаётся последовательность из неотрицательных чисел
    # от 0 до 65535, которую необходимо сжать.
    # Функция возвращает один любой объект encoding_table
    # и строку из нулей и единиц encoded_text.
    # Функция должна закодировать текст text кодом Хаффмана.

    encoding_table = {}  # здесь можно хранить объект любого типа
                         # например, можно хранить словарь
    encoded_text = ''

    # --- Замените код ниже на реализацию кода Хаффмана для текста text ---

    encoded_text = '011000010110001001100011'

    # ---------------------------------------------------------------------

    # print()
    # pprint.pprint(encoding_table)
    # print()
    # print('Закодированное сообщение:')
    # print('\n'.join([''.join(line) for line in zip(*[iter(encoded_text[:1000])]*100)]))
    # print('...')

    return encoding_table, encoded_text


def huffman_decode(encoding_table, encoded_text):
    # Функция принимает на вход ваш объект encoding_table
    # и составленную вами строку из нулей и единиц encoded_text.
    # Функция должна восстановить исходный список чисел text.

    # --- Напишите здесь раскодирование сообщения, которое вы закодировали ---

    text = [ord('a'), ord('b'), ord('c')]

    # ------------------------------------------------------------------------

    return text


def lzw_encode(text):
    # В параметре text передаётся список из натуральных чисел от 0 до 65535, который необходимо сжать.
    # Функция возвращает один любой объект initial_info
    # и список из произвольных натуральных чисел encoded_text.
    # Функция должна закодировать текст text с помощью алгоритма Лемпеля--Зива--Велча.

    set_ = set()
    for symbol in text:
        set_.add(symbol)
    d = {}
    i = 0
    for symbol in set_:
        d[(symbol,)] = i
        i += 1

    d_start = {}
    for j in d:
        d_start[d[j]] = j

    buffer = ()
    code = []
    for symbol in text:
        buffer += (symbol,)
        if buffer not in d:
            code.append(d[buffer[:-1]])
            d[buffer] = i
            i += 1
            buffer = (buffer[-1],)
    code.append(d[buffer])

    return d_start, code


def lzw_decode(d, code):
    # Функция принимает на вход составленный вами произвольный объект initial_info
    # и список из натуральных чисел encoded_text.
    # Функция должна восстановить исходный текст text.

    text = []
    text += d[code[0]]
    buffer = ()
    buffer += d[code[0]]
    i = len(d)
    for num in code[1:]:
        if num in d:
            text.extend(d[num])
            d[i] = buffer + (d[num][0],)
            i += 1
            buffer = d[num]
        else:
            text.extend(buffer + (buffer[0],))
            d[i] = buffer + (buffer[0],)
            i += 1
            buffer += (buffer[0],)
    print(text[:100])        

    return text


# def lzw_huffman_encode(text):
#     initial_info, encoded_text = lzw_encode(text)
#     encoding_table, encoded_text = huffman_encode(encoded_text)
#     return (initial_info, encoding_table), encoded_text


# def lzw_huffman_decode(obj, encoded_text):
#     initial_info, encoding_table = obj
#     encoded_text = huffman_decode(encoding_table, encoded_text)
#     return lzw_decode(initial_info, encoded_text)


def get_file_size(filename):
    import os
    return os.path.getsize(filename)


def get_encoded_filename(filename, method):
    return '{filename}.{method}'.format(**locals())


def get_decoded_filename(filename, method):
    return '{filename}.{method}.decoded.txt'.format(**locals())


def digits_to_bytes(digits):
    from itertools import zip_longest

    last_element_size = len(digits) % 8
    if last_element_size == 0:
        last_element_size = 8
    return last_element_size, bytes([int(''.join(t), 2) 
            for t in zip_longest(*([iter(digits)] * 8), fillvalue='0')])


def int_to_binary(i):
    return '{0:08b}'.format(i)


def bytes_to_digits(arg):
    last_element_size, bytes_ = arg

    return ''.join([int_to_binary(i) for i in bytes_[:-1]]
                        + [int_to_binary(bytes_[-1])[:last_element_size]])


def str_to_ints(s):
    return [ord(symbol) for symbol in s]


def ints_to_str(a):
    return ''.join([chr(i) for i in a])


def ints_to_bytes(ints):
    assert all(isinstance(x, int) and x >= 0 for x in ints)
    max_int = max(ints)
    bits_per_int = 1
    while 2 ** bits_per_int <= max_int:
        bits_per_int += 1

    int_to_bin = lambda x: ('{0:0' + str(bits_per_int) + 'b}').format(x)

    digits = ''.join([int_to_bin(code) for code in ints])

    return bits_per_int, digits_to_bytes(digits)


def bytes_to_ints(arg):
    bits_per_int, bytes_ = arg
    digits = bytes_to_digits(bytes_)

    return [int(digits[bits_per_int * i:bits_per_int * (i + 1)], 2) 
            for i in range(len(digits) // bits_per_int)]


def run_single_file(filename, title):
    global results

    source_size_bytes = get_file_size(filename)

    print('Считываем файл {0}: {3} ({1} байт, {2} килобайт)'
            .format(filename, source_size_bytes, ceil(source_size_bytes / 1024), title))

    text = str_to_ints(open(filename, 'r', encoding='utf-8').read())
    
    print('[готово]')
    print()

    for method, is_ready in IS_READY_METHOD:
        if is_ready:
            encode_method_name = '{0}_encode'.format(method)
            decode_method_name = '{0}_decode'.format(method)

            print('Кодируем текст с помощью вашей функции {0}()'.format(encode_method_name))
            
            helper_object, encoded_text = globals()[encode_method_name](text)
            if isinstance(encoded_text, list):
                bytes_ = ints_to_bytes(encoded_text)
            else:
                bytes_ = digits_to_bytes(encoded_text)
            
            print('[готово]')
            print()
            

            print('Записываем результат кодирования в файл {0}'.format(get_encoded_filename(filename, method)))

            encoded_file = open(get_encoded_filename(filename, method), 'bw')
            pickle.dump(helper_object, encoded_file)
            pickle.dump(bytes_, encoded_file)
            encoded_file.close()

            print('[готово]')
            print()


            enc_size_bytes = get_file_size(get_encoded_filename(filename, method))

            print('Считываем файл {0} ({1} байт, {2} килобайт)'
                    .format(get_encoded_filename(filename, method), enc_size_bytes, ceil(enc_size_bytes / 1024)))

            encoded_file = open(get_encoded_filename(filename, method), 'br')
            helper_object = pickle.load(encoded_file)
            if method == 'lzw':
                encoded_text = bytes_to_ints(pickle.load(encoded_file))
            else:
                encoded_text = bytes_to_digits(pickle.load(encoded_file))                
            
            print('[готово]')
            print()


            print('Декодируем текст с помощью вашей функции {0}()'.format(decode_method_name))

            decoded_text = globals()[decode_method_name](helper_object, encoded_text)

            print('[готово]')
            print() 


            print('Записываем результат декодирования в файл {0}'.format(get_decoded_filename(filename, method)))

            decoded_file = open(get_decoded_filename(filename, method), 'w', encoding='utf-8')
            print(ints_to_str(decoded_text), file=decoded_file, end='')
            decoded_file.close()

            print('[готово]')
            print()


            dec_size_bytes = get_file_size(get_decoded_filename(filename, method))

            print('Декодированный файл {0} ({1} байт, {2} килобайт)'
                    .format(get_decoded_filename(filename, method), dec_size_bytes, ceil(dec_size_bytes / 1024)))

            compression_ratio = round(100 * enc_size_bytes / source_size_bytes)

            if text == decoded_text:
                print('Исходный текст и декодированный текст совпадают. Счастье, радость ^_^')
                results.append((filename, method,'^_^, {0}%'.format(compression_ratio)))
            else:
                print('Декодированный текст не равен исходному. Печаль, уныние T_T')
                results.append((filename, method,'T_T, {0}%'.format(compression_ratio)))

            print('Сжатый файл занимает {0}% исходного'.format(compression_ratio))
            print()
    print()
    print('-' * 50)
    print()


def is_ejudge_mode():
    return 'compression.in' in os.listdir('.')


def run_test():
    fin = open('compression.in', 'r', encoding='utf-8')
    method, text = fin.read().split('\n', 1)

    text = str_to_ints(text)

    encode_method_name = '{0}_encode'.format(method)
    decode_method_name = '{0}_decode'.format(method)

    helper_object, encoded_text = globals()[encode_method_name](text)
    
    decoded_text = globals()[decode_method_name](helper_object, encoded_text)

    if text == decoded_text:
        print('^_^')
    else:
        print('T_T')


if __name__ == '__main__':
    results = []
    if is_ejudge_mode():
        run_test()
    else:
        for filename, title in SOURCE_FILES:
            run_single_file(filename, title)
        for filename, method, result in results:
            print('{0:40}'.format('{filename}, {method}:'
                                .format(**locals())) + result)


Считываем файл 01.txt: Test 1 (13 байт, 1 килобайт)
[готово]

Кодируем текст с помощью вашей функции lzw_encode()
[готово]

Записываем результат кодирования в файл 01.txt.lzw
[готово]

Считываем файл 01.txt.lzw (102 байт, 1 килобайт)
[готово]

Декодируем текст с помощью вашей функции lzw_decode()
[72, 101, 108, 108, 111, 44, 32, 119, 111, 114, 108, 100, 33]
[готово]

Записываем результат декодирования в файл 01.txt.lzw.decoded.txt
[готово]

Декодированный файл 01.txt.lzw.decoded.txt (13 байт, 1 килобайт)
Исходный текст и декодированный текст совпадают. Счастье, радость ^_^
Сжатый файл занимает 785% исходного


--------------------------------------------------

Считываем файл 02.txt: Test 2 (22 байт, 1 килобайт)
[готово]

Кодируем текст с помощью вашей функции lzw_encode()
[готово]

Записываем результат кодирования в файл 02.txt.lzw
[готово]

Считываем файл 02.txt.lzw (51 байт, 1 килобайт)
[готово]

Декодируем текст с помощью вашей функции lzw_decode()
[97, 97, 97, 97, 97, 97, 97, 97, 